In [1]:
import utils
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import GRU, Flatten, Concatenate,ELU,Permute,Dropout,MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping
import os
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Reshape, Bidirectional, Conv2D, MaxPooling2D, BatchNormalization, Dense, Flatten, Input, ZeroPadding2D
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from keras.metrics import AUC, F1Score, Precision, Accuracy
from sklearn.metrics import accuracy_score



c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


# CRNN

In [2]:
checkpoint_filepath = r'D:\VNtraditionalmusicclassification\data\checkpoint'

def CRNN(input_tensor=None,include_top=True):
    # Determine proper input shape
    if K.image_data_format() == 'channels_first':
        input_shape = (1, 96, 1366)
    else:
        input_shape = (96, 1366, 1)

    if input_tensor is None:
        melgram_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            melgram_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            melgram_input = input_tensor

    # Determine input axis
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
        freq_axis = 2
        time_axis = 3
    else:
        channel_axis = 3
        freq_axis = 1
        time_axis = 2

    # Input block
    x = ZeroPadding2D(padding=(0, 37))(melgram_input)
    x = BatchNormalization(axis=freq_axis, name='bn_0_freq')(x)

    # Conv block 1
    x = Conv2D(64, 3, 3, padding='same', name='conv1' )(x)
    x = BatchNormalization(axis=channel_axis, name='bn1')(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool1')(x)
    x = Dropout(0.2, name='dropout1')(x)

    # Conv block 2
    x = Conv2D(128, 3, 3, padding='same', name='conv2')(x)
    x = BatchNormalization(axis=channel_axis, name='bn2')(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(3, 3), name='pool2')(x)
    x = Dropout(0.2, name='dropout2')(x)

    # Conv block 3
    x = Conv2D(128, 3, 3, padding='same', name='conv3')(x)
    x = BatchNormalization(axis=channel_axis, name='bn3')(x)
    x = ELU()(x)
    x = MaxPooling2D((4,4), padding='same')(x)
    x = Dropout(0.2, name='dropout3')(x)

    # Conv block 4
    x = Conv2D(128, 3, 3, padding='same', name='conv4')(x)
    x = BatchNormalization(axis=channel_axis, name='bn4')(x)
    x = ELU()(x)
    x = MaxPooling2D((4,4), padding='same')(x)
    x = Dropout(0.2, name='dropout4')(x)

    # reshaping
    if K.image_data_format() == 'channels_last':
        x = Permute((3, 1, 2))(x)
    x = Reshape((1,1, 128))(x)
    # # Reshape to squeeze out the unnecessary dimensions
    x = Reshape((-1, 128))(x)  # Assuming the last dimension is 128

    # GRU block 1, 2, output
    x = GRU(32, return_sequences=True, name='gru1')(x)
    x = GRU(32, return_sequences=False, name='gru2')(x)
    x = Dropout(0.3)(x)
    if include_top:
        x = Dense(5, activation='sigmoid', name='output')(x)

    # Create model
    model = Model(melgram_input, x)
    if not os.path.exists(checkpoint_filepath + '/lcrnn'):
        os.makedirs(checkpoint_filepath + '/lcrnn')
    checkpoint1= tf.keras.callbacks.ModelCheckpoint(
    filepath= checkpoint_filepath + '/lcrnn' + '/lcrnn_{epoch:02d}_{val_accuracy:.4f}.weights.h5',
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=True,
    verbose=1
    )
    return model,checkpoint1

In [3]:
early = EarlyStopping(monitor='loss',
    patience= 10,
    verbose= 0,
    mode='auto',
    baseline= None,
    restore_best_weights= True)


In [4]:
# Directory containing audio files
audio_dir_ori = r'D:\VNtraditionalmusicclassification\data\VNTM3'

# Process audio files and retrieve log-mel spectrograms and labels
data, labels = utils.process_audio_files_crnn(audio_dir_ori)

In [5]:

# Concatenate the data
reshaped_data = np.transpose(data, (0, 2, 3, 1))

# Split data into training and combined validation-evaluation set
X_train, X_combined, y_train, y_combined = train_test_split(reshaped_data, labels, test_size=0.2, random_state=42, shuffle=True, stratify =labels)

# Split combined set into validation and evaluation sets
X_val, X_eval, y_val, y_eval = train_test_split(X_combined, y_combined, test_size=0.5, random_state=42, shuffle=True, stratify =y_combined)

In [6]:

# Label binarization
label_binarizer = LabelBinarizer()
y_train_one_hot_crnn = label_binarizer.fit_transform(y_train)
y_val_one_hot_crnn = label_binarizer.transform(y_val)
y_eval_one_hot_crnn = label_binarizer.transform(y_eval)



In [7]:
modelcrnn = tf.keras.Sequential()
modelcrnn ,checkpoint1= CRNN(include_top=True)

optimizer = Adam(learning_rate=0.001)
modelcrnn.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                   metrics=['accuracy'])
modelcrnn.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 96, 1366, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding2d (ZeroPadding2D)  │ (None, 96, 1440, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_0_freq (BatchNormalization)  │ (None, 96, 1440, 1)    │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 32, 480, 64)    │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn1 (BatchNormalization)        │ (None, 32, 480, 64)    │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ elu (ELU)                       │ (None, 32, 480, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1 (MaxPooling2D)            │ (None, 16, 240, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout1 (Dropout)              │ (None, 16, 240, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 6, 80, 128)     │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn2 (BatchNormalization)        │ (None, 6, 80, 128)     │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ elu_1 (ELU)                     │ (None, 6, 80, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool2 (MaxPooling2D)            │ (None, 2, 26, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout2 (Dropout)              │ (None, 2, 26, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3 (Conv2D)                  │ (None, 1, 9, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn3 (BatchNormalization)        │ (None, 1, 9, 128)      │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ elu_2 (ELU)                     │ (None, 1, 9, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 1, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout3 (Dropout)              │ (None, 1, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv4 (Conv2D)                  │ (None, 1, 1, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn4 (BatchNormalization)        │ (None, 1, 1, 128)      │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ elu_3 (ELU)                     │ (None, 1, 1, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 1, 1, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout4 (Dropout)              │ (None, 1, 1, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ permute (Permute)               │ (None, 128, 1, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 1, 1, 128)      │             

 Total params: 393,893 (1.50 MB)

 Trainable params: 392,805 (1.50 MB)

 Non-trainable params: 1,088 (4.25 KB)

In [8]:
modelcrnn.fit(X_train, y_train_one_hot_crnn, batch_size=32, epochs=100, validation_data=(X_val, y_val_one_hot_crnn), callbacks=[checkpoint1, early], shuffle=True)


Epoch 1/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 358ms/step - accuracy: 0.3825 - loss: 0.6220
Epoch 1: val_accuracy improved from -inf to 0.20000, saving model to D:\VNtraditionalmusicclassification\data\checkpoint/lcrnn/lcrnn_01_0.2000.weights.h5
63/63 ━━━━━━━━━━━━━━━━━━━━ 27s 374ms/step - accuracy: 0.3831 - loss: 0.6209 - val_accuracy: 0.2000 - val_loss: 0.5595
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step - accuracy: 0.5836 - loss: 0.4010
Epoch 2: val_accuracy did not improve from 0.20000
63/63 ━━━━━━━━━━━━━━━━━━━━ 23s 372ms/step - accuracy: 0.5844 - loss: 0.4005 - val_accuracy: 0.2000 - val_loss: 0.5996
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step - accuracy: 0.7520 - loss: 0.2875
Epoch 3: val_accuracy improved from 0.20000 to 0.52800, saving model to D:\VNtraditionalmusicclassification\data\checkpoint/lcrnn/lcrnn_03_0.5280.weights.h5
63/63 ━━━━━━━━━━━━━━━━━━━━ 24s 374ms/step - accuracy: 0.7523 - loss: 0.2872 - val_accuracy: 0.5280 - val_loss: 0.4757
Epoch 4/100
63/63 ━━━━━

In [9]:
# Evaluate the model
predict = modelcrnn.predict(X_eval)
# Calculate AUC ROC
auc_roc = AUC()
auc_roc.update_state(y_eval_one_hot_crnn, predict)
print("AUC ROC:", auc_roc.result().numpy())

# Calculate F1 Score
f1 = F1Score()
f1.update_state(y_eval_one_hot_crnn, predict)
print("F1 Score:", np.mean(f1.result().numpy()))

# Calculate Precision
pre = Precision()
pre.update_state(y_eval_one_hot_crnn, predict)
print("Precision Score", pre.result().numpy())

# Accuracy Score
acc = Accuracy()
acc.update_state(y_eval_one_hot_crnn, predict)
print("Accuracy Score", pre.result().numpy())

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 103ms/step
AUC ROC: 0.9868181
F1 Score: 0.9483639
Precision Score 0.9516129
Accuracy Score 0.9516129


In [10]:
predicted_labels = label_binarizer.inverse_transform(predict)
for i in range(len(predicted_labels)):
    if predicted_labels[i] == y_eval[i]:
        print(predicted_labels[i], y_eval[i])

cheo cheo
cailuong cailuong
cheo cheo
chauvan chauvan
catru catru
hatxam hatxam
cheo cheo
cailuong cailuong
hatxam hatxam
cailuong cailuong
cailuong cailuong
hatxam hatxam
chauvan chauvan
cheo cheo
cheo cheo
catru catru
cheo cheo
hatxam hatxam
hatxam hatxam
chauvan chauvan
catru catru
chauvan chauvan
hatxam hatxam
hatxam hatxam
chauvan chauvan
catru catru
chauvan chauvan
cheo cheo
cheo cheo
hatxam hatxam
chauvan chauvan
hatxam hatxam
hatxam hatxam
cailuong cailuong
cailuong cailuong
catru catru
chauvan chauvan
cheo cheo
chauvan chauvan
catru catru
hatxam hatxam
cheo cheo
cheo cheo
chauvan chauvan
hatxam hatxam
hatxam hatxam
hatxam hatxam
cheo cheo
chauvan chauvan
cailuong cailuong
cailuong cailuong
chauvan chauvan
catru catru
hatxam hatxam
chauvan chauvan
cailuong cailuong
catru catru
cailuong cailuong
cheo cheo
cheo cheo
catru catru
cailuong cailuong
catru catru
chauvan chauvan
catru catru
cailuong cailuong
catru catru
catru catru
cheo cheo
cheo cheo
hatxam hatxam
cailuong cailuong
ha

# PCRNN

In [2]:
checkpoint_filepath = r'D:\ProjectMusicGenre\data\checkpoint'
dataset_root =r'D:\ProjectMusicGenre\data'
saved_model_path = r'D:\ProjectMusicGenre\data\model'


In [7]:
if not os.path.exists(dataset_root + "/checkpoint"):
    os.makedirs(dataset_root + "/checkpoint")
if not os.path.exists(checkpoint_filepath):
    os.makedirs(checkpoint_filepath)
if not os.path.exists(checkpoint_filepath + '/pcrnn'):
    os.makedirs(checkpoint_filepath + '/pcrnn')

In [1]:
audio_dir = 'D:\ProjectMusicGenre\data\dataset10seconds'
stft_img, stft_data, labels = utils.process_audio_files(audio_dir)


In [5]:
print(stft_data[0].shape)

(513, 128)


In [4]:


# Split data into training and combined validation-evaluation set
img_train, img_conbined, stft_train, stft_combined, y_train, y_combined = train_test_split(stft_img, stft_data, labels, test_size=0.2, random_state=42, shuffle=True, stratify =labels)

# Split combined set into validation and evaluation sets
img_val, img_eval, stft_val, stft_eval, y_val, y_eval = train_test_split(img_conbined, stft_combined, y_combined, test_size=0.5, random_state=42, shuffle=True, stratify =y_combined)



In [5]:
img_train, img_val, img_eval = np.array(img_train), np.array(img_val), np.array(img_eval)
stft_train, stft_val, stft_eval = np.array(stft_train), np.array(stft_val), np.array(stft_eval)


In [6]:
n_class = 5
input_shape = (513, 128,3)
tf.random.set_seed(42)

In [7]:

def pcrnn(model,input_shape,rnn_input_shape):
    # Define input layer
    input_img = Input(shape=input_shape)

    # Define CNN Block
    cnn = Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same')(input_img)
    cnn = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(cnn)
    cnn = BatchNormalization()(cnn)
    cnn = Dropout(0.1, name='dropout1')(cnn)
    
    cnn = Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same')(cnn)
    cnn = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(cnn)
    cnn = BatchNormalization()(cnn)
    cnn = Dropout(0.1, name='dropout2')(cnn)

    cnn = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same')(cnn)
    cnn = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(cnn)
    cnn = BatchNormalization()(cnn)
    cnn = Dropout(0.1, name='dropout3')(cnn)

    cnn = Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same')(cnn)
    cnn = MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same')(cnn)
    cnn = BatchNormalization()(cnn)
    cnn = Dropout(0.1, name='dropout4')(cnn)

    cnn = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same')(cnn)
    cnn = MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same')(cnn)
    cnn = BatchNormalization()(cnn)
    cnn = Dropout(0.1, name='dropout5')(cnn)

    cnn = Flatten()(cnn)

    # Define Block 2
    rnn_input = Input(shape=rnn_input_shape)
    rnn = MaxPooling1D(pool_size=2,strides=2,data_format='channels_last')(rnn_input)
    rnn = Bidirectional(GRU(128))(rnn)
    rnn = Dropout(0.3)(rnn)


    # Concatenate CNN and RNN outputs
    concatenated_output = Concatenate()([cnn, rnn])

    # Define output layer
    output = Dense(5, activation='softmax')(concatenated_output)

    # Define the model
    model = Model(inputs=[input_img, rnn_input], outputs=output)
    if not os.path.exists(checkpoint_filepath + '/pcrnn'):
        os.makedirs(checkpoint_filepath + '/pcrnn')
    checkpoint1= tf.keras.callbacks.ModelCheckpoint(
    filepath= checkpoint_filepath + '/pcrnn' + '/pcrnn_{epoch:02d}_{val_accuracy:.4f}.weights.h5',
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=True,
    verbose=1
    )
    return model,checkpoint1

In [8]:
modelpcrnn = tf.keras.Sequential()
modelpcrnn, checkpoint1 = pcrnn(modelpcrnn, input_shape, rnn_input_shape=(513, 128))


In [9]:
modelpcrnn.compile(optimizer='adam',
                  loss='binary_crossentropy',
                   metrics=['accuracy'])
modelpcrnn.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 513, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 513, 128,  │        448 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 257, 64,   │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 257, 64,   │         64 │ max_pooling2d[0]… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout1 (Dropout)  │ (None, 257, 64,   │          0 │ batch_normalizat… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 257, 64,   │      4,640 │ dropout1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 129, 32,   │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 129, 32,   │        128 │ max_pooling2d_1[… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout2 (Dropout)  │ (None, 129, 32,   │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 129, 32,   │     18,496 │ dropout2[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 65, 16,    │          0 │ conv2d_2[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 65, 16,    │        256 │ max_pooling2d_2[… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout3 (Dropout)  │ (None, 65, 16,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 65, 16,    │     73,856 │ dropout3[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 17, 4,     │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 17, 4,     │        512 │ max_pooling2d_3[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout4 (Dropout)  │ (None, 17, 4,     │          0 │ batch_normalizat

 Total params: 373,477 (1.42 MB)

 Trainable params: 372,869 (1.42 MB)

 Non-trainable params: 608 (2.38 KB)

In [10]:
label_binarizer = LabelBinarizer()
y_train_one_hot = label_binarizer.fit_transform(y_train)
y_val_one_hot = label_binarizer.transform(y_val)
y_eval_one_hot = label_binarizer.transform(y_eval)

In [11]:
early = EarlyStopping(monitor='loss',
    patience= 5,
    verbose= 0,
    mode='auto',
    baseline= None,
    restore_best_weights= True)


In [13]:
history = modelpcrnn.fit(x=(img_train, stft_train),
                         y=y_train_one_hot,  
                         batch_size=32,
                         epochs=100,  
                         validation_data=((img_val,stft_val),y_val_one_hot),
                         callbacks=[checkpoint1, early]
)

Epoch 1/100
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step - accuracy: 0.6964 - loss: 0.2973
Epoch 1: val_accuracy improved from -inf to 0.58155, saving model to D:\ProjectMusicGenre\data\checkpoint/pcrnn/pcrnn_01_0.5816.weights.h5
187/187 ━━━━━━━━━━━━━━━━━━━━ 77s 375ms/step - accuracy: 0.6969 - loss: 0.2968 - val_accuracy: 0.5816 - val_loss: 0.3946
Epoch 2/100
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step - accuracy: 0.8890 - loss: 0.1348
Epoch 2: val_accuracy improved from 0.58155 to 0.83155, saving model to D:\ProjectMusicGenre\data\checkpoint/pcrnn/pcrnn_02_0.8316.weights.h5
187/187 ━━━━━━━━━━━━━━━━━━━━ 61s 328ms/step - accuracy: 0.8891 - loss: 0.1347 - val_accuracy: 0.8316 - val_loss: 0.1726
Epoch 3/100
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step - accuracy: 0.9350 - loss: 0.0916
Epoch 3: val_accuracy improved from 0.83155 to 0.88235, saving model to D:\ProjectMusicGenre\data\checkpoint/pcrnn/pcrnn_03_0.8824.weights.h5
187/187 ━━━━━━━━━━━━━━━━━━━━ 63s 334ms/step - accuracy: 0.9350 - loss:

In [14]:

# Evaluate the model
predict = modelpcrnn.predict([img_eval, stft_eval])
predicted_labels = label_binarizer.inverse_transform(predict).tolist()

# Calculate AUC ROC
auc_roc = AUC()
auc_roc.update_state(y_eval_one_hot, predict)
print("AUC ROC:", auc_roc.result().numpy())

# Calculate F1 Score
f1 = F1Score()
f1.update_state(y_eval_one_hot, predict)
print("F1 Score:", np.mean(f1.result().numpy()))

# Calculate Precision
pre = Precision()
pre.update_state(y_eval_one_hot, predict)
print("Precision Score", pre.result().numpy())

# Accuracy Score
acc = accuracy_score(y_eval, predicted_labels)
print("Accuracy Score", acc)


24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 90ms/step
AUC ROC: 0.9960858
F1 Score: 0.9852867
Precision Score 0.9853138
Accuracy Score 0.9853137516688919
